# SEBIR BESS test 
The purpose of this Python Notebook file is to test if the optimized Swing Equation Based Inertial Response (SEBIR) controller topology for a grid following Battery Energy Storage System (BESS) is able to mimic the frequency response of a given conventional generator in a wide range of conditions. Each test is conducted in the IEE 39 Bus System using the corresponding cases (base case and SEBIR case). All of the inputs to this file are entered into the first block of code, so this is the only part of the code that should be modified. The second block sets up the PowerFactory for simulations, and each of the following blocks represents a test case. 

The tests that this file conducts are:
* Load 25: 50% step increase in active power consumption. 
* Load 25: 100% step increase in active power consumption.
* Load 08: 50% step increase in active power consumption.
* Load 25: 50% step decrease in active power consumption.
* Load 25: 100% ramp increase in active power consumption over 60 seconds.
* Load 25: 50% step increase in active power consumption, BESS: Change initial SOC from 50% to 60%
* Load 25: 50% step increase in active power consumption, AGC: enabled


All other parameters are set to constant values, including the controller values which are set to the given optimized values. 

### Inputs
The following code block contains all the inputs that the user may want to modify.

NOTE: To run a different scenario, all you need to do is change the data underneath "Scenario information"!

In [ ]:
 # Import packages
import pandas as pd
import simulationfunctions as simfunc
import optimization as opt
import numpy as np
import plotly.express as px
from scipy.optimize import direct, Bounds
from sklearn.metrics import mean_squared_error
import sys

# Scenario information
gen_data_name = "G 03" # Name of generator being replaced ("G 03" or "G 10")
SC_connected = False # Set to true if a synchronous condenser is connected in the BESS file

# Information of the generator being replaced by the BESS
proj_name_base = r"\mattm\Base cases\39 Bus System - BASE (AGC)" # Base case file path

# Battery pack information
BESS_series_cells = 585 # Number of series cells in the battery pack (integer)

# SEBIR BESS information
proj_name_BESS_G03 =  r"\mattm\SEBIR cases\39 Bus System - SEBIR BESS at G03 (AGC)" # BESS G03 case file path
proj_name_BESS_G10 =  r"\mattm\SEBIR cases\39 Bus System - SEBIR BESS at G10 (AGC)" # BESS G10 case file path
BESS_data_name = 'PWM Converter' # Name of BESS converter in excel data file
BESS_power_name = 'm:Psum:busac in MW' # Name of BESS power column in excel data file
BESS_freq_model = "Frequency droop.ElmDsl"  # Name of frequency controller model 
BESS_PWM_model = "PWM Converter.ElmVscmono" # Name of PWM model (for changing power output)
BESS_PQ_model = "PQ Control.ElmDsl" # Name of PQ controller model
BESS_battery_model = "Battery Model 13Ah.ElmDsl"
BESS_DC_bus_model = "DC Bus.ElmTerm"
BESS_DC_source_model = "DC - Voltage Source.ElmDcu"

# Enter the optimal solutions for the different cases
optimal_solution_G03 = [0.24143677, 0.12300412, 0.10205761, 0.00163855] 
optimal_solution_G03_SC = [0.234120815, 'N/A', 0.266666667, 0.041833333] 
optimal_solution_G10 = [0.0470439, 0.01650206, 0.13504801, 7.3340192]
optimal_solution_G10_SC = [0.046632373,	'N/A', 0.18761114, 9.46926282] 

# Synchronous condenser information
SC_data_name_G03 = 'SynCon G03' # Name of SC replacing G03 in excel data file
SC_data_name_G10 = 'SynCon G10' # Name of SC replacing G10 in excel data file
SC_model_G03 = 'SynCon G03.ElmSym' # Model of SC replacing G03
SC_model_G10 = 'SynCon G10.ElmSym'  # Model of SC replacing G10

# AGC information 
AGC_model = 'AGC DSL Model.ElmDsl'

# Simulation setup 
sim_start_time = -1000 # Simulation start time (ms)
sim_stop_time = 300 # Simulation end time 
digsilent_path = r"C:\Program Files\DIgSILENT\PowerFactory 2023 SP2\Python\3.10" # DIgSILENT files path

# General results information
freq_data_name = "Bus 30" # Name of bus the frequency is being measured from in excel data file
base_results_path =  r"C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\simulation_results_BASE.csv" # Base case results file path
BESS_results_path =  r"C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\simulation_results_BESS.csv" # BESS case results file path
error_results_path = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\error_results.xlsx' # Error quantities results file path

# Test 1 results information
freq_image_path1 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 1\FreqResp.png' # Frequency response image file path
p_image_path1 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 1\GenP.png' # Power output image file path

# Test 2 results information
freq_image_path2 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 2\FreqResp.png' # Frequency response image file path
p_image_path2 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 2\GenP.png' # Power output image file path

# Test 3 results information
freq_image_path3 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 3\FreqResp.png' # Frequency response image file path
p_image_path3 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 3\GenP.png' # Power output image file path

# Test 4 results information
freq_image_path4 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 4\FreqResp.png' # Frequency response image file path
p_image_path4 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 4\GenP.png' # Power output image file path

# Test 5 results information
freq_image_path5 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 5\FreqResp.png' # Frequency response image file path
p_image_path5 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 5\GenP.png' # Power output image file path

# Test 6 results information
freq_image_path6 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 6\FreqResp.png' # Frequency response image file path
p_image_path6 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 6\GenP.png' # Power output image file path

# Test 7 results information
freq_image_path7 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 7\FreqResp.png' # Frequency response image file path
p_image_path7 = r'C:\Users\mattm\OneDrive - The University of Melbourne\Master of Electrical Engineering\Capstone\Python\Final scripts\Results\Result output\Tests\Test 7\GenP.png' # Power output image file path

### Simulation set up 
The following code block opens PowerFactory so that simulations can be conducted. 

In [ ]:
# Open PowerFactory
sys.path.append(digsilent_path) 
import powerfactory as pf
try:
    app = pf.GetApplicationExt()
    print(app) # PowerFactory will print below if there are no errors
except pf.ExitError as error: 
    print(error)
    print('error.code = %d' % error.code)

### Calculate the BESS parameters 
The battery pack parameters are calculated based on the known cell data and the number of series cells. It is assumed that the BESS needs to have the same MVA rating as the generator is replacing, so the number of parallel cells needed to achieve this rating can be calculated based on this. This indirectly determines the capacity value of the battery. As the C-rate of these cells is 3C, the capacity will always be 1/3 of the generator rating. The capacity could be chosen arbitrarily, but it is sensible to have the battery have the same maximum power output as the generator it is replacing. 

The control parameters are set to the optimal values as defined.

In [ ]:
# Set up BESS case corresponding to the generator being replaced
if(gen_data_name == "G 03"):
    proj_name_BESS = proj_name_BESS_G03
    SC_model = SC_model_G03
    SC_data_name = SC_data_name_G03
    generator_rated_power = 800 # Rating of the generator in MVA
    generator_inertia = 4.475 # Inertia of the generator in s (G10 = 4.2s, G03 = 4.475s)
    gen_setpoint = 650 # Active power setpoint of the generator in MW (G10 = 250 MW, G03 = 650 MW)
    if(SC_connected):
        optimal_solution = optimal_solution_G03_SC
    else:
        optimal_solution = optimal_solution_G03
    AGC_gain = 0.8425 # AGC gain for BESS

if(gen_data_name == "G 10"):
    proj_name_BESS = proj_name_BESS_G10
    SC_model = SC_model_G10
    SC_data_name = SC_data_name_G10
    generator_rated_power = 1000 # Rating of the generator in MVA
    generator_inertia = 4.2 # Inertia of the generator in s (G10 = 4.2s, G03 = 4.475s)
    gen_setpoint = 250 # Active power setpoint of the generator in MW (G10 = 250 MW, G03 = 650 MW)
    if(SC_connected):
        optimal_solution = optimal_solution_G10_SC
    else:
        optimal_solution = optimal_solution_G10
    AGC_gain = 21.25 # AGC gain for BESS
        
# Battery pack constants
cell_voltage = 2.228 # nominal voltage of one cell in V
c_rate = 0.5 # C rate of the cell (max current)
cell_capacity = 13 # Capacity rating of one cell in Ah 

# BESS parameters 
BESS_setpoint = gen_setpoint # BESS power setpoint (set to same as conventional generator it is replacing)
BESS_rated_power =  generator_rated_power # BESS rated power (set to same as conventional generator it is replacing)

# BESS calcualtions
BESS_DC_voltage = BESS_series_cells * cell_voltage # Total DC voltage of the battery pack in V
BESS_par_cells = round((BESS_rated_power*10**6)/ (cell_capacity * c_rate * BESS_DC_voltage)) # Number of parallel cells needed to achieve desired power rating 
BESS_capacity = cell_capacity * BESS_par_cells * BESS_DC_voltage/ (10**6) # Capacity of the battery pack in MWh

# Set optimal parameters 
droop_value, inertia_value, P_value, T_value = optimal_solution

### Define simulation test function
Define a function for running the tests.

In [ ]:
def run_test(test_number, error_results_path, freq_image_path, p_image_path, app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base, base_results_path, freq_data_name, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, BESS_PQ_model, BESS_results_path, SC_model, BESS_data_name, SC_data_name,gen_data_name, BESS_power_name, droop_value, inertia_value, P_value, T_value, SC_connected, AGC_model, AGC_connected, AGC_gain):
    # Run base case
    Rms, oInit, elmres = simfunc.base_case_setup(app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base)
    simfunc.toggle_AGC(app, AGC_model, AGC_connected) # Toggle AGC
    dfs_b = simfunc.base_case_sim(app, event_time, base_results_path, freq_data_name, Rms, oInit, elmres)
    
    # Run BESS case
    Rms, oInit, elmres = simfunc.SEBIR_case_setup(app, event, event_time, load_change, SC_connected, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, SC_model)
    simfunc.toggle_AGC(app, AGC_model, AGC_connected) # Toggle AGC
    simfunc.set_AGC_gain(app, BESS_PQ_model, AGC_gain) # Adjust gain of AGC
    dfs_BESS = simfunc.SEBIR_case_sim(droop_value, inertia_value, P_value, T_value, SC_connected, app, event_time, BESS_freq_model, BESS_PQ_model, BESS_results_path, freq_data_name, Rms, oInit, elmres)

    # Filter data frames to exclude data before t = 0
    dfs_BESS_filtered = dfs_BESS[dfs_BESS[("All calculations", "b:tnow in s")] >= 0]
    dfs_b_filtered = dfs_b[dfs_b[("All calculations", "b:tnow in s")] >= 0]

    # Find the RMS error of the power output responses
    if(SC_connected):
        power_rms_error = mean_squared_error(dfs_BESS_filtered[(BESS_data_name,BESS_power_name)]+ dfs_BESS_filtered[(SC_data_name,'m:Psum:bus1 in MW')], dfs_b_filtered[(gen_data_name,'m:Psum:bus1 in MW')], squared = False) 
    else:
        power_rms_error = mean_squared_error(dfs_BESS_filtered[(BESS_data_name,BESS_power_name)], dfs_b_filtered[(gen_data_name,'m:Psum:bus1 in MW')], squared = False) 
    print("Power RMS error:", power_rms_error)

    # Find the RMS error of the frequency responses
    frequency_rms_error = mean_squared_error(dfs_BESS_filtered[(freq_data_name,"m:fehz in Hz")], dfs_b_filtered[(freq_data_name,"m:fehz in Hz")], squared = False) # Frequency RMS error
    print("Frequency RMS error:", frequency_rms_error)

    # Create or load a DataFrame to store the results
    try:
        result_df = pd.read_excel(error_results_path)
    except FileNotFoundError:
        result_df = pd.DataFrame(columns=['Test Number', 'Power RMSE', 'Frequency RMSE', 'Overvoltage Warnings', 'Undervoltage Warnings'])

    # Create empty lists to store bus names with voltage warnings
    overvoltage_warnings = []
    undervoltage_warnings = []

    # Save the names of the buses where there was a voltage violation
    for col_name in dfs_BESS.columns[1:]:
        if "m:u in p.u." in col_name:
            if dfs_BESS[col_name].max() > 1.06:
                overvoltage_warnings.append(col_name[0])

            if dfs_BESS[col_name].min() < 0.94:
                undervoltage_warnings.append(col_name[0])

    # Convert the voltage warnings lists to comma-separated strings
    overvoltage_warnings_str = ', '.join(overvoltage_warnings)
    undervoltage_warnings_str = ', '.join(undervoltage_warnings)

    # Check if the 'test_number' already exists in the DataFrame
    existing_index = result_df[result_df['Test Number'] == test_number].index

    # Create DataFrames for error values and voltage warnings
    error_results = pd.DataFrame({
        'Test Number': [test_number],
        'Power RMSE': [power_rms_error],
        'Frequency RMSE': [frequency_rms_error],
        'Overvoltage Warnings': [overvoltage_warnings_str],
        'Undervoltage Warnings': [undervoltage_warnings_str]
    })

    # If 'test_number' already exists, replace the existing row; otherwise, concatenate the DataFrames
    if not existing_index.empty:
        result_df.loc[existing_index] = error_results.values[0]
    else:
        result_df = pd.concat([result_df, error_results], ignore_index=True)

    # Export the updated results to an Excel file
    result_df.to_excel(error_results_path, index=False)

    # Save diagrams 
    # Generate and save plots for the report based on the desired length of the data
    data_length = 300 # Period of the simulation you want to actually plot (seconds)
    simfunc.plot_freq(dfs_b_filtered, dfs_BESS_filtered, freq_data_name, freq_image_path, data_length)
    simfunc.plot_power(dfs_b_filtered, dfs_BESS_filtered, gen_data_name, BESS_data_name, BESS_power_name, SC_data_name, SC_connected, p_image_path, data_length)
    return

### Test 1
* Load 25: 50% step increase in active power consumption 

In [ ]:
# Test 1 info
test_number = 1
event = 0 # Event type: "0" = load step, "1" = load ramp
load_change = 50 # Step change for load event (%) 
load_model_name = "Load 25.ElmLod" # Name of load you want to change
gen_model_name = "None" # Name of generator you want to switch off

# Default values
event_time = 5 # Event time in s
BESS_soc0 = 0.5 # Intial SOC of the battery
AGC_connected = False # Set to true if AGC you want to enable AGC

# Run test
run_test(test_number, error_results_path, freq_image_path1, p_image_path1, app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base, base_results_path, freq_data_name, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, BESS_PQ_model, BESS_results_path, SC_model, BESS_data_name, SC_data_name,gen_data_name, BESS_power_name, droop_value, inertia_value, P_value, T_value, SC_connected, AGC_model, AGC_connected, AGC_gain)

### Test 2
* Load 25: 100% step increase in active power consumption 

In [ ]:
# Test 2 info
test_number = 2
event = 0 # Event type: "0" = load step, "1" = load ramp
load_change = 100 # Step change for load event (%) 
load_model_name = "Load 25.ElmLod" # Name of load you want to change
gen_model_name = "None" # Name of generator you want to switch off

# Default values
event_time = 5 # Event time in s
BESS_soc0 = 0.5 # Intial SOC of the battery
AGC_connected = False # Set to true if AGC you want to enable AGC

# Run test
run_test(test_number, error_results_path, freq_image_path2, p_image_path2, app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base, base_results_path, freq_data_name, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, BESS_PQ_model, BESS_results_path, SC_model, BESS_data_name, SC_data_name,gen_data_name, BESS_power_name, droop_value, inertia_value, P_value, T_value, SC_connected, AGC_model, AGC_connected, AGC_gain)

### Test 3
* Load 08: 50% step increase in active power consumption 

In [ ]:
# Test 3 info
test_number = 3
event = 0 # Event type: "0" = load step, "1" = load ramp
load_change = 50 # Step change for load event (%) 
load_model_name = "Load 08.ElmLod" # Name of load you want to change
gen_model_name = "None" # Name of generator you want to switch off

# Default values
event_time = 5 # Event time in s
BESS_soc0 = 0.5 # Intial SOC of the battery
AGC_connected = False # Set to true if AGC you want to enable AGC

# Run test
run_test(test_number, error_results_path, freq_image_path3, p_image_path3, app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base, base_results_path, freq_data_name, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, BESS_PQ_model, BESS_results_path, SC_model, BESS_data_name, SC_data_name,gen_data_name, BESS_power_name, droop_value, inertia_value, P_value, T_value, SC_connected, AGC_model, AGC_connected, AGC_gain)

### Test 4
* Load 25: 50% step decrease in active power consumption.

In [ ]:
# Test 4 info
test_number = 4
event = 0 # Event type: "0" = load step, "1" = load ramp
load_change = -50 # Step change for load event (%) 
load_model_name = "Load 25.ElmLod" # Name of load you want to change
gen_model_name = "None" # Name of generator you want to switch off

# Default values
event_time = 5 # Event time in s
BESS_soc0 = 0.5 # Intial SOC of the battery
AGC_connected = False # Set to true if AGC you want to enable AGC

# Run test
run_test(test_number, error_results_path, freq_image_path4, p_image_path4, app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base, base_results_path, freq_data_name, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, BESS_PQ_model, BESS_results_path, SC_model, BESS_data_name, SC_data_name,gen_data_name, BESS_power_name, droop_value, inertia_value, P_value, T_value, SC_connected, AGC_model, AGC_connected, AGC_gain)

### Test 5
* Load 25: 100% ramp increase in active power consumption over 60 seconds.

In [ ]:
# Test 5 info
test_number = 5
event = 1 # Event type: "0" = load step, "1" = load ramp
load_change = 100 # Ramp increase for load event (%) 
load_model_name = "Load 25.ElmLod" # Name of load you want to change
gen_model_name = "None" # Name of generator you want to switch off

# Default values
event_time = 5 # Event time in s
BESS_soc0 = 0.5 # Intial SOC of the battery
AGC_connected = False # Set to true if AGC you want to enable AGC

# Run test
run_test(test_number, error_results_path, freq_image_path5, p_image_path5, app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base, base_results_path, freq_data_name, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, BESS_PQ_model, BESS_results_path, SC_model, BESS_data_name, SC_data_name,gen_data_name, BESS_power_name, droop_value, inertia_value, P_value, T_value, SC_connected, AGC_model, AGC_connected, AGC_gain)

### Test 6
* Load 25: 50% step increase in active power consumption
* BESS: Change initial SOC from 50% to 60%

In [ ]:
# Test 6 info
test_number = 6
event = 0 # Event type: "0" = load step, "1" = load ramp
load_change = 50 # Step change for load event (%) 
load_model_name = "Load 25.ElmLod" # Name of load you want to change
gen_model_name = "None" # Name of generator you want to switch off
BESS_soc0 = 0.6 # Inital SOC

# Default values
event_time = 5 # Event time in s
AGC_connected = False # Set to true if AGC you want to enable AGC

# Run test
run_test(test_number, error_results_path, freq_image_path6, p_image_path6, app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base, base_results_path, freq_data_name, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, BESS_PQ_model, BESS_results_path, SC_model, BESS_data_name, SC_data_name,gen_data_name, BESS_power_name, droop_value, inertia_value, P_value, T_value, SC_connected, AGC_model, AGC_connected, AGC_gain)

### Test 7
* Load 25: 50% step increase in active power consumption
* System: AGC enabled

In [ ]:
# Test 7 info
test_number = 7
event = 0 # Event type: "0" = load step, "1" = load ramp
load_change = 50 # Step change for load event (%) 
load_model_name = "Load 25.ElmLod" # Name of load you want to change
gen_model_name = "None" # Name of generator you want to switch off
AGC_connected = True # Set to true if AGC you want to enable AGC

# Default values
event_time = 5 # Event time in s
BESS_soc0 = 0.5 # Intial SOC of the battery

# Run test
run_test(test_number, error_results_path, freq_image_path7, p_image_path7, app, event, event_time, load_change, load_model_name, gen_model_name, sim_start_time, sim_stop_time, proj_name_base, base_results_path, freq_data_name, proj_name_BESS, BESS_series_cells, BESS_par_cells, BESS_DC_voltage, BESS_soc0, BESS_rated_power, BESS_setpoint, BESS_battery_model, BESS_PWM_model, BESS_DC_bus_model, BESS_DC_source_model, BESS_freq_model, BESS_PQ_model, BESS_results_path, SC_model, BESS_data_name, SC_data_name,gen_data_name, BESS_power_name, droop_value, inertia_value, P_value, T_value, SC_connected, AGC_model, AGC_connected, AGC_gain)